In [1]:
from google.colab import drive
drive.mount('/content/drive')
#запустить

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 10.4 MB/s eta 0:00:00


In [3]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.2 MB/s eta 0:00:00


In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))



Found GPU at: /device:GPU:0


In [5]:
import os
import pandas as pd
import gc
import numpy as np
#запустить


from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer


In [6]:
def reduce_mem_usage(data):
    start_memory = data.memory_usage().sum() / 1024**2
    print(f"Initial memory usage: {start_memory:.2f} MB")

    int_type_dict = {
        (np.iinfo(np.int8).min,  np.iinfo(np.int8).max):  np.int8,
        (np.iinfo(np.int16).min, np.iinfo(np.int16).max): np.int16,
        (np.iinfo(np.int32).min, np.iinfo(np.int32).max): np.int32,
        (np.iinfo(np.int64).min, np.iinfo(np.int64).max): np.int64,
    }

    float_type_dict = {
        (np.finfo(np.float16).min, np.finfo(np.float16).max): np.float16,
        (np.finfo(np.float32).min, np.finfo(np.float32).max): np.float32,
        (np.finfo(np.float64).min, np.finfo(np.float64).max): np.float64,
    }

    data.columns = data.columns.str.strip()  # Remove leading/trailing whitespace from column names

    for column in data.columns:
        col_type = data[column].dtype

        if np.issubdtype(col_type, np.integer):
            c_min = data[column].min()
            c_max = data[column].max()
            dtype = next((v for k, v in int_type_dict.items() if k[0] <= c_min and k[1] >= c_max), None)
            if dtype:
                data[column] = data[column].astype(dtype)
        elif np.issubdtype(col_type, np.floating):
            c_min = data[column].min()
            c_max = data[column].max()
            dtype = next((v for k, v in float_type_dict.items() if k[0] <= c_min and k[1] >= c_max), None)
            if dtype:
                data[column] = data[column].astype(dtype)
        elif col_type == object:
            data[column] = data[column].astype('category')

    end_memory = data.memory_usage().sum() / 1024**2
    print(f"Final memory usage: {end_memory:.2f} MB")
    print(f"Reduced by {(start_memory - end_memory) / start_memory * 100:.1f}%")
    return data


In [7]:
def prepare_transactions_dataset(path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=1,
                                 save_to_path=None, verbose: bool=False):
    """
    возвращает готовый pd.DataFrame с признаками, на которых можно учить модель для целевой задачи.
    path_to_dataset: str
        путь до датасета с партициями
    num_parts_to_preprocess_at_once: int
        количество партиций, которые будут одновременно держаться в памяти и обрабатываться
    num_parts_total: int
        общее количество партиций, которые нужно обработать
    save_to_path: str
        путь до папки, в которой будет сохранен каждый обработанный блок в .parquet формате. Если None, то не будет сохранен
    verbose: bool
        логирует каждый обрабатываемый кусок данных
    """
    preprocessed_frames = []
    for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                                   desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once,
                                                             verbose=verbose)

        #print(transactions_frame.columns.values)
        print(transactions_frame.isnull().sum())

        feature_cols = list(transactions_frame.columns.values)
        feature_cols.remove('id')
        feature_cols.remove('rn')


        dummies = pd.get_dummies(transactions_frame[feature_cols], columns=feature_cols)
        dummy = dummies.columns.values

        ohe_features = pd.concat([transactions_frame, dummies], axis=1)
        ohe_features = ohe_features.drop(columns=feature_cols)


        ohe_features.groupby("id")
        features = ohe_features.groupby("id")[dummy].sum().reset_index(drop=False)

        transactions_frame = features.merge(pd.read_csv('train_target.csv'),how='inner', on='id')



        #записываем подготовленные данные в файл
        if save_to_path:
            block_as_str = str(step)
            if len(block_as_str) == 1:
                block_as_str = '00' + block_as_str
            else:
                block_as_str = '0' + block_as_str
            transactions_frame.to_parquet(os.path.join(save_to_path, f'processed_chunk_{block_as_str}.parquet'))

        preprocessed_frames.append(transactions_frame)
    return pd.concat(preprocessed_frames)


In [8]:
data = read_and_prepare_dataset(path_to_dataset='/content/drive/MyDrive/Skillbox/Skillbox/train_data', num_parts_to_preprocess_at_once=1, num_parts_total=3)


In [ ]:
data.columns.values

array(['enc_paym_8_1', 'pre_maxover2limit_2', 'pre_over2limit_2',
       'pre_since_opened_2', 'pre_loans5_13', 'flag',
       'pre_loans_outstanding_5', 'enc_paym_20_1', 'enc_paym_10_0',
       'pre_loans_credit_limit_7', 'is_zero_loans90_1',
       'pre_loans_credit_cost_rate_10', 'pre_loans_max_overdue_sum_2',
       'pre_loans_total_overdue_1', 'pre_since_opened_17', 'enc_paym_9_1',
       'pre_maxover2limit_14', 'enc_paym_7_2',
       'pre_loans_credit_cost_rate_5', 'pre_loans_outstanding_3',
       'enc_paym_0_2', 'enc_paym_17_0', 'pre_util_17', 'enc_paym_3_0',
       'enc_loans_credit_type_3', 'enc_paym_16_3', 'pre_till_pclose_6',
       'pre_fterm_3', 'pre_loans5_9', 'pre_since_confirmed_16',
       'pre_loans_credit_cost_rate_7', 'enc_paym_19_3', 'pre_loans530_14',
       'pre_loans530_0', 'enc_paym_10_3', 'pre_loans530_9',
       'enc_loans_credit_status_5', 'enc_paym_6_0', 'pre_pterm_3',
       'pre_since_confirmed_5', 'enc_loans_account_holder_type_0',
       'enc_paym_14_0

In [9]:
X = data.drop('flag', axis=1)
y = data['flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [10]:
target_test = y_test.to_csv('/content/drive/MyDrive/pipeline_data/target_test.csv')
data_test = X_test.to_csv('/content/drive/MyDrive/pipeline_data/data_test.csv')

In [ ]:
from tensorflow.python.util import deprecation
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

def objective(trial):
    param = {
        'objective': 'binary:logistic',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True)
    }

    if param['booster'] == 'gbtree' or param['booster'] == 'dart':
        param['max_depth'] = trial.suggest_int('max_depth', 1, 20)
        param['eta'] = trial.suggest_float('eta', 1e-8, 1.0, log=True)
        param['gamma'] = trial.suggest_float('gamma', 1e-8, 1.0, log=True)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
        param['tree_method'] = 'gpu_hist'

    bst = xgb.train(param, xgb.DMatrix(X_train, label=y_train))
    preds = bst.predict(xgb.DMatrix(X_valid))

    roc_auc = roc_auc_score(y_valid, preds)

    return  roc_auc



study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print(study.best_trial)


[I 2023-08-11 13:51:44,395] A new study created in memory with name: no-name-b95b5790-042e-4c20-bbe0-b64a06951c9e
[I 2023-08-11 13:52:00,744] Trial 0 finished with value: 0.6883868942149609 and parameters: {'booster': 'gblinear', 'lambda': 0.07081108492657667, 'alpha': 3.730857154545405e-08}. Best is trial 0 with value: 0.6883868942149609.
[I 2023-08-11 13:52:10,901] Trial 1 finished with value: 0.6528738075798155 and parameters: {'booster': 'dart', 'lambda': 1.2180861351226252e-08, 'alpha': 1.3310883736437917e-07, 'max_depth': 17, 'eta': 9.295880184273267e-06, 'gamma': 0.002254448936624998, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.6883868942149609.
[I 2023-08-11 13:52:22,744] Trial 2 finished with value: 0.6456452357636495 and parameters: {'booster': 'gbtree', 'lambda': 7.334591192899664e-05, 'alpha': 1.3679976059709723e-07, 'max_depth': 18, 'eta': 0.00012414053210758872, 'gamma': 1.961333483442345e-08, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.68838

FrozenTrial(number=64, state=TrialState.COMPLETE, values=[0.7418904361402966], datetime_start=datetime.datetime(2023, 8, 11, 14, 1, 22, 358140), datetime_complete=datetime.datetime(2023, 8, 11, 14, 1, 29, 824953), params={'booster': 'gbtree', 'lambda': 4.349033644826225e-08, 'alpha': 2.5932770408822093e-07, 'max_depth': 8, 'eta': 0.4302705603015105, 'gamma': 0.5328364271638715, 'grow_policy': 'depthwise'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'alpha': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'max_depth': IntDistribution(high=20, log=False, low=1, step=1), 'eta': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'gamma': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'grow_policy': CategoricalDistribution(choices=('depthwise', 'lossguide'))}, trial_id=64, value=Non

In [ ]:
import lightgbm as lgb


# Загрузите демонстрационные данные (оставим то же разделение на обучение и валидацию)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
    }

    if param["boosting_type"] == "goss":
        param["top_rate"] = trial.suggest_float("top_rate", 0.0, 1.0)
        param["other_rate"] = trial.suggest_float("other_rate", 0.0, 1.0 - param["top_rate"])

    # Добавление оптимизации весов классов
    weight_0 = trial.suggest_float("weight_0", 0.1, 100)
    weight_1 = trial.suggest_float("weight_1", 0.1, 100)
    class_weight = {0: weight_0, 1: weight_1}

    gbm = lgb.train(param, lgb.Dataset(X_train, y_train, weight=[class_weight[i] for i in y_train]),
                    valid_sets=[lgb.Dataset(X_valid, y_valid, weight=[class_weight[i] for i in y_valid])], verbose_eval=False)

    preds = gbm.predict(X_valid)
    roc_auc = roc_auc_score(y_valid, preds)

    return roc_auc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print(study.best_trial)




[I 2023-08-11 16:42:04,131] A new study created in memory with name: no-name-1b792dfb-5edf-4846-bfb8-56debd1286aa
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-08-11 16:42:58,196] Trial 0 finished with value: 0.7509196516352888 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 227, 'learning_rate': 0.012426405129439365, 'min_child_samples': 54, 'reg_alpha': 0.00013980402892206065, 'reg_lambda': 3.1119352795237193e-07, 'colsample_bytree': 0.5975129685608177, 'subsample': 0.9122438323431739, 'weight_0': 13.3378746807292, 'weight_1': 94.41976972136601}. Best is trial 0 with value: 0.7509196516352888.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbo

FrozenTrial(number=19, state=TrialState.COMPLETE, values=[0.7600499550917132], datetime_start=datetime.datetime(2023, 8, 11, 16, 57, 16, 934982), datetime_complete=datetime.datetime(2023, 8, 11, 16, 58, 1, 32566), params={'boosting_type': 'gbdt', 'num_leaves': 244, 'learning_rate': 0.05523810956569323, 'min_child_samples': 76, 'reg_alpha': 4.694550524777625e-06, 'reg_lambda': 0.00027763856733398104, 'colsample_bytree': 0.6791657438084778, 'subsample': 0.6588516462860272, 'weight_0': 38.74633910795909, 'weight_1': 68.0173914405459}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('gbdt', 'dart', 'goss')), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'learning_rate': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'min_child_samples': IntDistribution(high=100, log=False, low=5, step=1), 'reg_alpha': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'reg_lambda': FloatDi

In [ ]:

from catboost import CatBoostClassifier, Pool

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_strength': trial.suggest_float('random_strength', 0, 1),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2, 30),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS', 'No'])
    }

    if param['bootstrap_type'] == 'Bayesian':
        param['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)

    if param['bootstrap_type'] == 'Bernoulli':
        param['subsample'] = trial.suggest_float('subsample', 0.1, 1)

    param['task_type'] = 'GPU'



    model = CatBoostClassifier(**param, verbose=False)
    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=False)

    preds = model.predict_proba(X_valid)[:, 1]
    roc_auc = roc_auc_score(y_valid, preds)

    return roc_auc



study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
   print(f"{key}: {value}")

[I 2023-08-11 15:44:43,874] A new study created in memory with name: no-name-56e0bb5e-5e3b-4044-b132-c079d47fc3f1
[I 2023-08-11 15:44:53,894] Trial 0 finished with value: 0.7569938130987225 and parameters: {'iterations': 816, 'depth': 6, 'learning_rate': 0.19942742174100073, 'random_strength': 0.8532417830505427, 'l2_leaf_reg': 24.68440503556323, 'border_count': 92, 'bootstrap_type': 'No'}. Best is trial 0 with value: 0.7569938130987225.
[I 2023-08-11 15:45:02,429] Trial 1 finished with value: 0.7502117067770935 and parameters: {'iterations': 278, 'depth': 10, 'learning_rate': 0.27852866783428976, 'random_strength': 0.2876586681132518, 'l2_leaf_reg': 22.098356051034916, 'border_count': 223, 'bootstrap_type': 'Bernoulli', 'subsample': 0.7813747630912954}. Best is trial 0 with value: 0.7569938130987225.
[I 2023-08-11 15:45:07,802] Trial 2 finished with value: 0.7434583106278124 and parameters: {'iterations': 179, 'depth': 5, 'learning_rate': 0.15047291429319948, 'random_strength': 0.6392

Number of finished trials:  200
Best trial:
Value:  0.7605264114964814
Params: 
iterations: 768
depth: 10
learning_rate: 0.06552457960440336
random_strength: 0.2991397355158974
l2_leaf_reg: 4.281272265260282
border_count: 173
bootstrap_type: No
